In [7]:
import pickle, numpy as np, keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import models

/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
with open('/data/jrgillick/x_train.pkl') as f:
    X_train = pickle.load(f)

In [3]:
with open('/data/jrgillick/x_test.pkl') as f:
    X_test = pickle.load(f)

In [4]:
with open('/data/jrgillick/y_train.pkl') as f:
    y_train = pickle.load(f)

In [5]:
with open('/data/jrgillick/y_test.pkl') as f:
    y_test = pickle.load(f)

In [333]:
reload(models)

<module 'models' from 'models.py'>

In [897]:
with open('/data/jrgillick/x_train_deltas.pkl','rb') as f:
    X_train_deltas = pickle.load(f)
    
with open('/data/jrgillick/x_test_deltas.pkl','rb') as f:
    X_test_deltas = pickle.load(f)

with open('/data/jrgillick/y_train_deltas.pkl','rb') as f:
    y_train_deltas = pickle.load(f)
    
with open('/data/jrgillick/y_test_deltas.pkl','rb') as f:
    y_test_deltas = pickle.load(f)

(30531, 3, 91)

30531

In [898]:
X_train_deltas.shape

(28974, 2304)

In [831]:
deltas_model = models.initialize_feed_forward_model(input_dim=1536,layer_sizes=[60,30])

In [836]:
deltas_model.fit(X_train_deltas,np.array(y_train_deltas),batch_size=128,epochs=3)

Epoch 1/3
29617/29617 [==============================] - 2s - loss: 0.5939 - acc: 0.6853     
Epoch 2/3
29617/29617 [==============================] - 2s - loss: 0.5848 - acc: 0.6949     
Epoch 3/3
29617/29617 [==============================] - 2s - loss: 0.5763 - acc: 0.6990     


In [837]:
deltas_model.evaluate(X_test_deltas,np.array(y_test_deltas),batch_size=128)

7405/7405 [==============================] - 0s     


[0.6052891886049151, 0.6737339633610665]

In [735]:
models.evaluate_model(deltas_model, X_test_deltas, y_test_deltas,model_type='keras')

7488/7534 [============================>.] - ETA: 0sAccuracy: 0.672 +/- 0.011 (5065/7534) | Precision: 0.670 | Recall: 0.676 | F1: 0.673


(0.6722856384390762,
 0.67001055966209078,
 0.67553899387809424,
 0.67276341948310148)

In [446]:
X_train.shape

(30905, 4800)

In [20]:
model = models.train_cv_logistic_regression(X_train,y_train)

In [21]:
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.634 +/- 0.011 (4899/7727) | Precision: 0.636 | Recall: 0.621 | F1: 0.628


(0.634010612139252,
 0.63587969124301302,
 0.62068069628474931,
 0.62818827241651332)

In [3]:
def balance(X,y):
    X,y = shuffle(X,y)
    positive_indices = [i for i,lab in enumerate(y) if lab == 1]
    negative_indices = [i for i,lab in enumerate(y) if lab == 0]
    negative_indices = negative_indices[0:len(positive_indices)]
    X = [x for i,x in enumerate(X) if i in positive_indices or i in negative_indices]
    y = [x for i,x in enumerate(y) if i in positive_indices or i in negative_indices]
    return (X,y)

In [462]:
scaler = StandardScaler()
scaler.fit(X_train)
#model.fit(scaler.transform(X_train),y_train,batch_size=128,epochs=1)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [1]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

NameError: name 'scaler' is not defined

In [283]:
with open('/data/jrgillick/less_skip_thought_feats.pkl','rb') as f:
    skip_thought_feats = pickle.load(f)

In [284]:
with open('/data/jrgillick/less_labs.pkl','rb') as f:
    all_labs= pickle.load(f)

In [62]:
#skip_thought_feats = skip_thought_feats[0:215] + skip_thought_feats[216:]
#all_labs = all_labs[0:215] + all_labs[216:]

In [63]:
#lstm_inputs_by_speech = []
#for i,f in enumerate(skip_thought_feats):
#    try:
#        lstm_inputs_by_speech.append(models.format_lstm_input(f,input_size=4800,lstm_length=3))
#    except:
#        print i

In [344]:
lstm_X, lstm_y = models.format_lstm_input(skip_thought_feats,all_labs,input_size=4800,lstm_length=6)

100%|██████████| 302/302 [00:00<00:00, 530880.05it/s]
0it [00:00, ?it/s]
100%|██████████| 127/127 [00:00<00:00, 450276.08it/s]


In [345]:
np.vstack(lstm_X).shape

(55548, 6, 4800)

In [800]:
960/5.*2

384.0

In [50]:
#lstm_labs = [lab[3:] for lab in all_labs]
#lstm_X = np.vstack(lstm_inputs_by_speech)
#lstm_y = np.hstack([np.hstack(lstm_labs[0:215]),np.hstack(lstm_labs[216:])])

In [347]:
balanced_x, balanced_y = models.balance(lstm_X,lstm_y)

In [348]:
balanced_x = np.array(balanced_x)
balanced_y = np.array(balanced_y)

In [349]:
shape = balanced_x.shape
balanced_x = balanced_x.reshape((shape[0],shape[2],shape[3]))

In [350]:
lstm_x_train, lstm_x_test = train_test_split(balanced_x,test_size=0.2, random_state=43)
lstm_y_train, lstm_y_test = train_test_split(balanced_y,test_size=0.2, random_state=43)

In [87]:
#conv_model = models.initialize_conv_model(4800,[6],dropout=0.5)
#conv_model.fit(lstm_x_train[:,2:,:],lstm_y_train,batch_size=128,epochs=1)
#conv_model.evaluate(lstm_x_test[:,2:,:],lstm_y_test,batch_size=128)

In [899]:
lstm_model = models.initialize_lstm_model(192,[80,40],bidirectional=False,dropout=0.3)

In [912]:
lstm_model.fit(X_train_deltas.reshape(X_train_deltas.shape[0],12,-1),np.array(y_train_deltas),batch_size=128,epochs=30,shuffle=True)

Epoch 1/30
28974/28974 [==============================] - 13s - loss: 0.5276 - acc: 0.7343    
Epoch 2/30
28974/28974 [==============================] - 13s - loss: 0.5236 - acc: 0.7379    
Epoch 3/30
28974/28974 [==============================] - 13s - loss: 0.5216 - acc: 0.7420    
Epoch 4/30
28974/28974 [==============================] - 13s - loss: 0.5198 - acc: 0.7437    
Epoch 5/30
28974/28974 [==============================] - 13s - loss: 0.5173 - acc: 0.7440    
Epoch 6/30
28974/28974 [==============================] - 13s - loss: 0.5209 - acc: 0.7413    
Epoch 7/30
28974/28974 [==============================] - 13s - loss: 0.5146 - acc: 0.7444    
Epoch 8/30
28974/28974 [==============================] - 13s - loss: 0.5115 - acc: 0.7460    
Epoch 9/30
28974/28974 [==============================] - 13s - loss: 0.5127 - acc: 0.7450    
Epoch 10/30
28974/28974 [==============================] - 13s - loss: 0.5083 - acc: 0.7509    
Epoch 11/30
28974/28974 [========================

In [914]:
lstm_model.evaluate(X_test_deltas.reshape(X_test_deltas.shape[0],12,-1),np.array(y_test_deltas),batch_size=128)

7168/7244 [============================>.] - ETA: 0s

[0.65942188098508303, 0.70223633387943185]

In [915]:
models.evaluate_model(lstm_model,X_test_deltas.reshape(X_test_deltas.shape[0],12,-1),np.array(y_test_deltas),model_type='keras')

7200/7244 [============================>.] - ETA: 0sAccuracy: 0.702 +/- 0.011 (5087/7244) | Precision: 0.711 | Recall: 0.693 | F1: 0.702


(0.7022363335173937,
 0.71132867132867128,
 0.6932933478735005,
 0.70219522297390591)

In [464]:
clf = models.train_cv_logistic_regression(X_train,y_train)

In [ ]:
models.evaluate_model(clf,X_test,y_test)

In [469]:
(len(X_train) + len(X_test))

38632

In [173]:
model.evaluate(scaler.transform(X_test),y_test,batch_size=128)

7168/7727 [==========================>...] - ETA: 0s

[0.63626607806281155, 0.63905784936282617]

In [175]:
models.evaluate_model(model,scaler.transform(X_test),y_test,model_type='keras')

7456/7727 [===========================>..] - ETA: 0sAccuracy: 0.639 +/- 0.011 (4938/7727) | Precision: 0.638 | Recall: 0.637 | F1: 0.637


(0.6390578491005565,
 0.63787721123829344,
 0.63704858404780462,
 0.63746262836344736)

In [111]:
models.evaluate_model(lstm_model,lstm_x_test,lstm_y_test,model_type='keras')

7456/7633 [============================>.] - ETA: 0sAccuracy: 0.618 +/- 0.011 (4720/7633) | Precision: 0.641 | Recall: 0.555 | F1: 0.595


(0.6183676143063016,
 0.64088729016786572,
 0.55489229172073706,
 0.59479760745583521)